In [ ]:
# pip install tensorflow==2.10

In [ ]:
# pip install keras

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from joblib import dump, load

import pandas as pd
import numpy as np

In [38]:
df = pd.read_csv('HF_stages.txt', sep=",")
df = df.dropna()
df.head()

,Apatient,AStages,AAcebutolol,AAcetazolamide,AAcetazolamide_Injection,AAmiloride,AAtenolol,AAzilsartan,ABenazepril,ABisoprolol,...,AR87,AR88,AR89,AR90,AR91,AR92,AR93,AR94,AR97,AR99
0,PT078431814,2.75,0,0,0,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,PT078628852,2.00,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,PT080357705,2.00,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,PT080489266,2.00,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,PT080504908,2.50,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0


In [39]:
for col in df.columns:
    print(col)

Apatient
AStages
AAcebutolol
AAcetazolamide
AAcetazolamide_Injection
AAmiloride
AAtenolol
AAzilsartan
ABenazepril
ABisoprolol
ABumetanide
ABumetanide_Injection
ACanagliflozin
ACandesartan
ACaptopril
ACarvedilol
AChlorothiazide
AChlorothiazide_Injection
AChlorthalidone
ACirculatory_Assist_Device
ADapagliflozin
ADefibrillator_Pacemaker
ADigoxin
ADigoxin_Injection
ADobutamine_Injection
ADopamine_Injection
AEmpagliflozin
AEnalapril
AEnalapril_Injection
AEpinephrine_Injection
AEplerenone
AErtugliflozin
AEsmolol_Injection
AEthacrynate
AFosinopril
AFurosemide
AFurosemide_Injection
AHeart_Transplant
AHospital_Stay
AHydralazine
AHydralazine_Injection
AHydrochlorothiazide
AIndapamide
AIntensive_Care
AIrbesartan
AIron_Injection
AIsosorbide
AIvabradine
ALabetalol
ALabetalol_Injection
ALisinopril
ALosartan
AMajor_Surgery
AMannitol_Injection
AMetolazone
AMetoprolol
AMetoprolol_Injection
AMilrinone_Injection
AMoexipril
ANadolol
ANebivolol
ANitroglycerin_Injection
ANitroprusside_Injection
ANorepinephr

In [23]:
#  df = df.iloc[:, 0:160]

In [40]:
for col in df.columns:
    print(col)

Apatient
AStages
AAcebutolol
AAcetazolamide
AAcetazolamide_Injection
AAmiloride
AAtenolol
AAzilsartan
ABenazepril
ABisoprolol
ABumetanide
ABumetanide_Injection
ACanagliflozin
ACandesartan
ACaptopril
ACarvedilol
AChlorothiazide
AChlorothiazide_Injection
AChlorthalidone
ACirculatory_Assist_Device
ADapagliflozin
ADefibrillator_Pacemaker
ADigoxin
ADigoxin_Injection
ADobutamine_Injection
ADopamine_Injection
AEmpagliflozin
AEnalapril
AEnalapril_Injection
AEpinephrine_Injection
AEplerenone
AErtugliflozin
AEsmolol_Injection
AEthacrynate
AFosinopril
AFurosemide
AFurosemide_Injection
AHeart_Transplant
AHospital_Stay
AHydralazine
AHydralazine_Injection
AHydrochlorothiazide
AIndapamide
AIntensive_Care
AIrbesartan
AIron_Injection
AIsosorbide
AIvabradine
ALabetalol
ALabetalol_Injection
ALisinopril
ALosartan
AMajor_Surgery
AMannitol_Injection
AMetolazone
AMetoprolol
AMetoprolol_Injection
AMilrinone_Injection
AMoexipril
ANadolol
ANebivolol
ANitroglycerin_Injection
ANitroprusside_Injection
ANorepinephr

In [41]:
df['AStages'] = df['AStages'].round(0)
    

In [42]:
df['AStages'].unique()

array([3., 2., 4., 1.])

In [43]:
df['AStages'] = df['AStages'].astype(object)

In [44]:
def prepareY(df):
  Y = df["AStages"]
  yencoder = LabelEncoder()
  yencoder.fit(Y)
  dump(yencoder,"yencoder.joblib")
  return yencoder.transform(Y)
y = prepareY(df)
df = df.drop(["AStages"], axis=1)
pd.DataFrame(y).head()

,0
0,2
1,1
2,1
3,1
4,1


In [45]:
round(
    pd.DataFrame(y).value_counts()/(2042+1085+826+223)*100
    )

1    49.0
2    26.0
0    20.0
3     5.0
Name: count, dtype: float64

In [46]:
df = df.drop(['Apatient'], axis=1)

In [ ]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OrdinalEncoder
# from sklearn.preprocessing import MinMaxScaler

# numerical_ix = df.select_dtypes(include=['int64', 'float64']).columns
# categorical_ix = df.select_dtypes(include=['object', 'bool']).columns

# column_trans = ColumnTransformer(
#    [('cat', OrdinalEncoder(),categorical_ix),
#     ('num', MinMaxScaler(feature_range=(-1, 1)), numerical_ix)],
#     remainder='drop')
# column_trans.fit(df)
# dump(column_trans,"column_trans.joblib")
# X = column_trans.transform(df)
# pd.DataFrame(X).head()

In [47]:
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=42)

In [48]:
yhot = np_utils.to_categorical(y)
yhot_train = np_utils.to_categorical(y_train)
yhot_test = np_utils.to_categorical(y_test)

In [53]:
# define baseline model
def baseline_model():
  # create model
  model = Sequential()
  
  # Rectified Linear Unit Activation Function
  model.add(Dense(2050, input_dim=1025, activation='relu'))
  model.add(Dense(2050, activation = 'relu'))
  # Softmax for multi-class classification
  model.add(Dense(4, activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [54]:
cmodel = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=100, verbose=0)

C:\Users\paulo\AppData\Local\Temp\ipykernel_3296\2991258205.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  cmodel = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=100, verbose=0)


In [55]:
kfold = KFold(n_splits=10, shuffle=True)

In [36]:
# result = cross_val_score(cmodel, df, yhot, cv=kfold)

# print("Result: %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

KeyboardInterrupt: 

In [56]:
model = baseline_model()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, yhot_train, validation_split=0.33, epochs=200, batch_size=100, verbose=0)

In [ ]:
import matplotlib.pyplot as plt

# list all data in history
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [58]:
# evaluate the keras model
_, accuracy = model.evaluate(X_test, yhot_test)
print('Accuracy from evaluate: %.2f' % (accuracy*100))

40/40 [==============================] - 0s 5ms/step - loss: 4.6568 - accuracy: 0.4366
Accuracy from evaluate: 43.66


In [59]:
predict_x = model.predict(X_test)
pred = np.argmax(predict_x, axis=1)
print(f'Prediction Accuracy: {(pred == y_test).mean() * 100:f}')

40/40 [==============================] - 0s 5ms/step
Prediction Accuracy: 43.655227


In [60]:
model_json = model.to_json()

with open("customermodel.json", "w") as json_file:
  json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [61]:
from keras.models import model_from_json

# load json and create model
json_file = open('customermodel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Loaded model from disk


In [62]:
loaded_model.summary()

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_93 (Dense)            (None, 2050)              2103300   
                                                                 
 dense_94 (Dense)            (None, 2050)              4204550   
                                                                 
 dense_95 (Dense)            (None, 4)                 8204      
                                                                 
Total params: 6,316,054
Trainable params: 6,316,054
Non-trainable params: 0
_________________________________________________________________


In [63]:
predict_x = loaded_model.predict(df)
pred = np.argmax(predict_x, axis=1)
print(f'Prediction Accuracy: {(pred == y).mean() * 100:f}')

131/131 [==============================] - 1s 5ms/step
Prediction Accuracy: 69.947318


In [64]:
pd.DataFrame(predict_x)

,0,1,2,3
0,3.903836e-04,9.987069e-01,8.965209e-04,6.226448e-06
1,1.525870e-06,5.778150e-07,9.999977e-01,1.922923e-07
2,4.632178e-07,9.999993e-01,2.207673e-07,1.527443e-10
3,5.976141e-10,9.999993e-01,7.112349e-07,2.029955e-10
4,4.595314e-04,1.727328e-03,9.690030e-01,2.881023e-02
...,...,...,...,...
4171,5.439996e-05,2.275811e-05,9.999207e-01,2.141978e-06
4172,2.318594e-08,7.743810e-08,9.999999e-01,4.028485e-08
4173,9.953207e-01,4.678496e-03,6.431294e-07,1.215734e-07
4174,5.537918e-07,3.040969e-07,1.075547e-06,9.999980e-01
